In [421]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum

from blowfish import Blowfish

In [422]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [423]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [424]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [425]:
data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
print(data_pk2_path)
print(media_pk2_path)

c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Data.pk2
c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Media.pk2


In [426]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [427]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [428]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [429]:
_data_file_stream = open(data_pk2_path, 'rb')

In [430]:
signature = _data_file_stream.read(30)
version =  struct.unpack('<i', _data_file_stream.read(4))[0]
encrypted = _data_file_stream.read(1)
encryption_checksum = _data_file_stream.read(16)
payload = _data_file_stream.read(205)


In [431]:
blowfish_checksum_decoded = "Joymax Pak File"

In [432]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [433]:
def read_block_at(position: int):
    _data_file_stream.seek(position, io.SEEK_SET)
    buffer = _data_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [434]:
Root = read_blocks_at(256)

In [435]:
blocks_in_memory = {
  "": Root
}

In [436]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336014543280781,
     'ModifyTime': 133336014543280781,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Compound',
     'CreationTime': 133336014543290789,
     'ModifyTime': 133336014543290789,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Dungeon',
     'CreationTime': 133336014545490716,
     'ModifyTime': 133336014545490716,
     'DataPosition': 87366,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.File: 2>,
     'Name': 'easteuropequest_soldier_masimus.bsr',
     'CreationTime': 133256738142034503,
     'ModifyTime': 131956339597547415,
     'DataPosition': 4043115,
     'DataSize': 848,
     'NextBlock': 0,
 

## regioninfo.txt

In [437]:
region_info_txt = "regioninfo.txt"

In [438]:
root = blocks_in_memory[""]
entry = None
for entries in root:
  entry = next((x for x in entries["Entries"] if x["Name"] == region_info_txt), None)
  if entry:
    break

entry

{'Type': <PackEntryType.File: 2>,
 'Name': 'regioninfo.txt',
 'CreationTime': 133287648859918505,
 'ModifyTime': 133287648859900000,
 'DataPosition': 2949956592,
 'DataSize': 149014,
 'NextBlock': 0,
 'Payload': b'\x00\x00'}

In [439]:
_data_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
buffer = io.BytesIO(_data_file_stream.read(entry["DataSize"]))


In [440]:
lines = buffer.readlines()

modern_region_info = {}

for line in lines:
  l = line.decode('utf-8', errors="replace").strip()
  parts = l.split('\t')
  region_info = {
    "Type": parts[0],
    "Name": parts[1],
    "XSector": int(parts[2]),
    "YSector": int(parts[3]),
    "RegionType": parts[4],
    "minX": float(parts[5]),
    "minY": float(parts[6]),
    "maxX": float(parts[7]),
    "maxY": float(parts[8]),
  }

  # modern_region_info[region_info["Name"]] = region_info


#TOWN	��������	182	96	ALL	0	0	0	0
#FIELD	��Ȳ����	1	128	donwhang	0	0	0	0
#FIELD	��Ȳ�����̺�Ʈ	9	128	donwhang_event	0	0	0	0
#FIELD	����Ȳ��456	2	128	jinsi	0	0	0	0
#FIELD	����Ȳ��456	3	128	jinsi	0	0	0	0
#FIELD	����Ȳ��456	4	128	jinsi	0	0	0	0
#FIELD	����Ȳ��3	5	128	jinsi	0	0	0	0
#FIELD	����Ȳ��12	6	128	jinsi	0	0	0	0
#FIELD	����Ȳ��12	7	128	jinsi	0	0	0	0
#TOWN	���_���	69	69	ALL	0	0	0	0
#TOWN	���_���	69	70	ALL	0	0	0	0
#TOWN	���_���	70	69	ALL	0	0	0	0
#TOWN	���_���	70	70	ALL	0	0	0	0
#TOWN	���_���	71	69	ALL	0	0	0	0
#TOWN	���_���	71	70	ALL	0	0	0	0
#TOWN	���_���	72	69	ALL	0	0	0	0
#TOWN	���_���	72	70	ALL	0	0	0	0
#TOWN	���_���	73	69	ALL	0	0	0	0
#TOWN	���_���	73	70	ALL	0	0	0	0
#FIELD	���_����ʵ�	69	67	ALL	0	0	0	0
#FIELD	���_����ʵ�	69	68	ALL	0	0	0	0
#FIELD	���_����ʵ�	70	67	ALL	0	0	0	0
#FIELD	���_����ʵ�	70	68	ALL	0	0	0	0
#FIELD	���_����ʵ�	71	67	ALL	0	0	0	0
#FIELD	���_����ʵ�	71	68	ALL	0	0	0	0
#FIELD	���_����ʵ�	72	67	ALL	0	0	0	0
#FIELD	���_����ʵ�	72	68	ALL	0	0	0	0
#FIELD	���_����ʵ�	73	67	ALL	0	0	0	0
#FIELD	���_�